In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
dataset = pd.read_csv('World Happiness Report 2022.csv')
dataset.head()
x_data = dataset.iloc[:, [3,4]].values

In [ ]:
from sklearn.cluster import KMeans

wcss = []

for i in range(1, 20):
    kmeans = KMeans(
        n_clusters = i,
    )

    kmeans.fit(x_data)

    wcss.append(kmeans.inertia_)

In [ ]:
plt.plot(range(1, 20), wcss)

In [ ]:
kmeans = KMeans(n_clusters = 5)
y_kmeans = kmeans.fit_predict(x_data)

In [ ]:
plt.scatter(x_data[y_kmeans == 0, 0], x_data[y_kmeans == 0, 1], s = 100, c = 'red', label = 'Cluster 1')
plt.scatter(x_data[y_kmeans == 1, 0], x_data[y_kmeans == 1, 1], s = 100, c = 'blue', label = 'Cluster 2')
plt.scatter(x_data[y_kmeans == 2, 0], x_data[y_kmeans == 2, 1], s = 100, c = 'green', label = 'Cluster 3')
plt.scatter(x_data[y_kmeans == 3, 0], x_data[y_kmeans == 3, 1], s = 100, c = 'cyan', label = 'Cluster 4')
plt.scatter(x_data[y_kmeans == 4, 0], x_data[y_kmeans == 4, 1], s = 100, c = 'magenta', label = 'Cluster 5')
plt.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1], s = 300, c = 'yellow', label = 'Centroids')
plt.title('Clusters of Hapiness')
plt.xlabel('Whisker-high')
plt.ylabel('Whisker-low')
plt.legend()
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, accuracy_score
from matplotlib.colors import ListedColormap
from sklearn.cluster import KMeans

# Carga el conjunto de datos
data = pd.read_csv('World Happiness Report 2022.csv')  # Reemplaza "tu_archivo.csv" con el nombre de tu archivo de datos

X = data[['Explained by: GDP per capita', 'Explained by: Social support']]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Realiza un bucle para probar diferentes números de clusters
wcss = []  # Lista para almacenar el "codo" de la suma de cuadrados
for i in range(1, 11):
    kmeans = KMeans(n_clusters=i, init='k-means++', random_state=42)
    kmeans.fit(X_scaled)
    wcss.append(kmeans.inertia_)  # Inertia es la suma de cuadrados dentro del cluster

# Grafica el método del codo para encontrar el número óptimo de clusters
plt.figure(figsize=(8, 6))
plt.plot(range(1, 11), wcss, marker='o', linestyle='--')
plt.xlabel('Número de clusters')
plt.ylabel('Inertia')
plt.title('Método del codo')
plt.show()


# Entrena el modelo K-Means con el número de clusters seleccionado
n_clusters = 3  # Reemplaza con el número de clusters seleccionado
kmeans = KMeans(n_clusters=n_clusters, init='k-means++', random_state=42)
kmeans.fit(X_scaled)

# Agrega las etiquetas de cluster al conjunto de datos original
data['Cluster'] = kmeans.labels_



In [ ]:
# Seleccionar las columnas relevantes
X = data[["Explained by: GDP per capita", "Explained by: Social support", "Explained by: Healthy life expectancy", "Explained by: Freedom to make life choices", "Explained by: Generosity", "Explained by: Perceptions of corruption"]]

from sklearn.cluster import KMeans

# Crear un modelo de k-means con 5 clústeres
kmeans = KMeans(n_clusters=5, random_state=0)

# Ajustar el modelo a los datos
kmeans.fit(X)

# Etiquetar los países con el clúster al que pertenecen
labels = kmeans.labels_

import plotly.express as px

# Agregar las etiquetas de los clústeres al DataFrame original
data['Cluster'] = labels

# Crear un mapa interactivo con colores representando los clústeres
fig = px.scatter_geo(data, locations="Country", color="Cluster", projection="natural earth")

# Mostrar el mapa
fig.show()

